## Reference 
https://huggingface.co/CHIH-HUNG/llama-2-13b-FINETUNE3_3.3w-r4-q_k_v_o
https://huggingface.co/datasets/huangyt/FINETUNE3
https://huggingface.co/erhwenkuo
https://huggingface.co/datasets/erhwenkuo/school_math_0.25m-zhtw
https://huggingface.co/datasets/erhwenkuo/medical_dialogue-chinese-zhtw
https://huggingface.co/datasets/DataAgent/medical-qa-instruction-zhtw
```
Model Card for Model ID
在llama-2-13b上使用huangyt/FINETUNE3資料集進行訓練，總資料筆數約3.3w

Fine-Tuning Information
GPU: RTX4090 (single core / 24564MiB)
model: meta-llama/Llama-2-13b-hf
dataset: huangyt/FINETUNE3 (共約3.3w筆訓練集)
peft_type: LoRA
lora_rank: 16
lora_target: q_proj, k_proj, v_proj, o_proj
per_device_train_batch_size: 8
gradient_accumulation_steps: 8
learning_rate : 4e-4
epoch: 1
precision: bf16
quantization: load_in_4bit

Fine-Tuning Detail
train_loss: 0.579
train_runtime: 4:6:11 (use deepspeed)
Evaluation
與Llama-2-13b比較4種Benchmark，包含ARC、HellaSwag、MMLU、TruthfulQA
評估結果使用本地所測的分數，並使用load_in_8bit
```
```
python finetune.py \
    --load_4bit \
    --base_model 'meta-llama/Llama-2-13b-hf' \
    --data_path 'yahma/alpaca-cleaned' \
    --output_dir './lora-alpaca' \
    --batch_size 8 \
    --micro_batch_size 4 \
    --num_epochs 1 \
    --learning_rate 4e-4 \
    --cutoff_len 512 \
    --val_set_size 2000 \
    --lora_r 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --lora_target_modules '[q_proj, k_proj, v_proj, o_proj]' \
    --train_on_inputs \
    --group_by_length
```

## A. KERNEL

## 036. DEMO Dataset 使用專用 library folder 之 image容器 kernel
<span style="color:red">Change to Default kernel:   Image_S_home_alpaca-lora_latest</span>.


### TRY IT
1. Lauch  a new notebook tab
2. Change to kernel:  Image_S_home_pytorch_2.1.0-cuda11.8-cudnn8-devel
3. In first cell add content below

```
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin:/usr/ubuntu_bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path
```

In [ ]:
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin:/usr/ubuntu_bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path

In [ ]:
pip install datasets -q

## 1. Loading dataset

In [ ]:
import json
from datasets import load_dataset

# 讀取數據集，take可以取得該數據集前n筆資料
dataset = load_dataset("DataAgent/medical-qa-instruction-zhtw", split="train", streaming=True)

# 提取所需欄位並建立新的字典列表
limit=30000
extracted_data = []
for example in dataset:
    extracted_example = {
        "instruction": example["instruction"],
        "input": example["input"],
        "output": example["output"]
    }
    extracted_data.append(extracted_example)
    if len(extracted_data) == limit:
        break

# 指定 JSON 文件名稱
json_filename = "medical-qa-instruction-zhtw.json"

# 寫入 JSON 文件
with open(json_filename, "w") as json_file:
    json.dump(extracted_data, json_file, indent=4)

print(f"數據已提取並保存為 {json_filename}")

In [ ]:
import pandas as pd
df = pd. read_json ( 'medical-qa-instruction-zhtw.json' )
df

In [ ]:
!rm -rf alpaca-lora
!cp -rf /work/u00cjz00/slurm_jobs/demo/alpaca-lora .

In [ ]:
!cp finetune.py alpaca-lora/

In [ ]:
%cd alpaca-lora

In [ ]:
!pip install -r requirements.txt -q
!pip install protobuf install accelerate==0.24.1 bitsandbytes transformers accelerate bitsandbytes bitsandbytes==0.41 scipy -q

In [ ]:
!python3 finetune.py \
    --base_model '/work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf' \
    --data_path '../medical-qa-instruction-zhtw.json' \
    --output_dir './lora-alpaca' \
    --batch_size 128 \
    --micro_batch_size 4 \
    --num_epochs 3 \
    --learning_rate 1e-4 \
    --cutoff_len 512 \
    --val_set_size 2000 \
    --lora_r 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --lora_target_modules '[q_proj,v_proj]' \
    --train_on_inputs \
    --group_by_length